<a href="https://colab.research.google.com/github/castudil/DEAP-OneMax/blob/main/DEAP_OneMax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Primero debemos instalar el framework DEAP que permite trabajar con algoritmos geneticos. DEAP es un acronimo que proviene del nombre Distributed Evolutionary Algorithms in Python.

In [1]:
!pip install deap

     |████████████████████████████████| 163kB 7.5MB/s 


Existen unos modulos de algoritmos geneticos pre programados.  debemos especificar en que consiste nuestro problema especifico, fitness, funcion de costo, como se compone el individuo, como se inicializa el individuo, etc. lo primero que haremos sera importar los modulos de deap que nos permitiran hacer esto.

In [13]:
import random

from deap import base
from deap import creator
from deap import tools

El problema ONeMAX

Este es el hola mundo del los algoritmos geneticos. Este problema es un ejemplo de juguete para aprender a ejecutar un GA simple.
consite en una secuencia de bits (ceros o unos) y buscamos el individuo cuya suma de bits sea maxima. obviamente una secuencia de unos es la que dara la mayor suma, sin embargo el computador no lo sabe. queremos que lo encuentre, a traves de los operadores de geneticos.

ejemplo:

10010 -> la suma de los digitos es 2

01110 -> la suma de los digitos es 3. Por lo tanto esta solucion es mejor que la anterior.

11111 -> la suma es 5. para individuos de largo 5 (o sea 5 bits) esta es la solucion óptima. esta solucion es unica (solo hay una secuencia de bit que tiene la mejor solucion) y su costo es conocido (es la suma de los bits, que en este caso es 5).



In [11]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


El mecanismo create es utilizado por DEAP para configurar elementos del GA. FitnessMax sera nuestra funcion de fitness, que utiliza la funcion de deap preprgoramada llamada base.fitness. ademas le asignamos una variable llamada peso (weight)

por otro lado Individual tiene el cromosoma de cada individuo. Es una lista en python que ademas se le asigna un fitness.

In [15]:
toolbox = base.Toolbox()

Toolbox es una clase importante en DEAP, tiene pre programado muchos operadores geneticos como cruzamientos, seleccion y mutación.

In [16]:
toolbox.register("cero_o_uno", random.randint, 0, 1)

cero_o_uno define un atributo del individuo. Es una especificacion del metodo randint. en el GA a cada atributo se le llama "gen". en este caso cada gen es un bit y puede asumir valores 0 o 1. En este caso lo implementamos como un valor entero. la funcion random.randomint permite inicializar el valor del gen de manera aleatoria entre un valor 0 o un 1.

In [17]:
# Structure initializers
#                         define 'individual' to be an individual
#                         consisting of 100 'attr_bool' elements ('genes')
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.cero_o_uno, 100)

este segemento de codigo inicializa a los individuos de la poblacion. crea una funcion llamada individual que genera un individuo de largo 100

In [18]:
# define the population to be a list of individuals
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

ahora se registra una funcion llamada population que genera una lista de individuos. En este punto aun no se especifica cuantos individuos habra en una generacion. ese numero se indica mas adelante en el codigo

In [19]:
# the goal ('fitness') function to be maximized
def evalOneMax(individual):
    return sum(individual),

para facilitar el calculo del fitness se define una funcion. esta es la funcion de costo que enterminaologia DEAP se conoce como `evaluacion`.

In [ ]:
#----------
# Operator registration
#----------
# register the goal / fitness function
toolbox.register("evaluate", evalOneMax)

se registra la funcion evaluate que internamente llama a la funcion en python evalOneMax que acabamos de definir

In [ ]:

# register the crossover operator
toolbox.register("mate", tools.cxTwoPoint)

mate significa cruzamiento. en este caso invocamos una funcion pre definida que se llama cxTwoPoint Crossover que fue explicada en clases. hay mucha informacion sobre ese operador de cruzamiento. solo hay que buscar en google o revisar la documentacion de DEAP


`def tools.cxTwoPoint(ind1, ind2)`
Executes a two-point crossover on the input sequence individuals. The two individuals are modified in place and both keep their original length.
:param ind1: The first individual participating in the crossover.
:param ind2: The second individual participating in the crossover.
:returns: A tuple of two individuals.
This function uses the ~random.randint function from the Python base random module.

In [ ]:
# register a mutation operator with a probability to
# flip each attribute/gene of 0.05
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

registra la funcion mutate que invoca al operador predefinido mutFlipBit que simplemente cambia un bit de valor.

In [ ]:
# operator for selecting individuals for breeding the next
# generation: each individual of the current generation
# is replaced by the 'fittest' (best) of three individuals
# drawn randomly from the current generation.
toolbox.register("select", tools.selTournament, tournsize=3)


se define la funcion select que indica el operador de seleccion. en este caso se usa seleccion por torneo.

A continuacion el loop principal del algoritmo

In [ ]:

#----------

def main():

  #semilla para la generacion seudo aleatoria
    random.seed(64)

    # create an initial population of 300 individuals (where
    # each individual is a list of integers)
    pop = toolbox.population(n=300)

    # CXPB  is the probability with which two individuals
    #       are crossed
    #
    # MUTPB is the probability for mutating an individual
    CXPB, MUTPB = 0.5, 0.2
    
    print("Start of evolution")
    
    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, pop))
    for ind, fit in zip(pop, fitnesses):
        ind.fitness.values = fit
    
    print("  Evaluated %i individuals" % len(pop))

    # Extracting all the fitnesses of 
    fits = [ind.fitness.values[0] for ind in pop]

    # Variable keeping track of the number of generations
    g = 0
    
    # Begin the evolution
    while max(fits) < 100 and g < 1000:
        # A new generation
        g = g + 1
        print("-- Generation %i --" % g)
        
        # Select the next generation individuals
        offspring = toolbox.select(pop, len(pop))
        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))
    
        # Apply crossover and mutation on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):

            # cross two individuals with probability CXPB
            if random.random() < CXPB:
                toolbox.mate(child1, child2)

                # fitness values of the children
                # must be recalculated later
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:

            # mutate an individual with probability MUTPB
            if random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values
    
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        print("  Evaluated %i individuals" % len(invalid_ind))
        
        # The population is entirely replaced by the offspring
        pop[:] = offspring
        
        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in pop]
        
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x*x for x in fits)
        std = abs(sum2 / length - mean**2)**0.5
        
        print("  Min %s" % min(fits))
        print("  Max %s" % max(fits))
        print("  Avg %s" % mean)
        print("  Std %s" % std)
    
    print("-- End of (successful) evolution --")
    
    best_ind = tools.selBest(pop, 1)[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))

if __name__ == "__main__":
    main()

# New Section